# Training Facenet-v1 model

##Verify Nvidia

Verify Nvidia for use GPU.

In [ ]:
!nvidia-smi

Thu Sep 18 16:58:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Mount drive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Preparing dataset

This Python 3 environment comes with many helpful analytics libraries installed

It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

For example, here's several helpful packages to load


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("quadeer15sh/celeba-face-recognition-triplets")

print("Path to dataset files:", path)

100%|██████████| 231M/231M [00:11<00:00, 21.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3


##Verify dataset

In [ ]:
!ls /root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3/

 CelebAAttrs.csv  'CelebA FR Triplets'


In [ ]:
!ls /root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3/CelebAAttrs.csv

/root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3/CelebAAttrs.csv


In [ ]:
!echo /root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3/CelebA FR Triplets

/root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3/CelebA FR Triplets


In [ ]:
for root, dirs, files in os.walk("/root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3"):
    for f in files[:10]:
        print(os.path.join(root, f))
    break

/root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3/CelebAAttrs.csv


Move to local drive

In [ ]:
!cp -r /root/.cache/kagglehub/datasets/quadeer15sh/celeba-face-recognition-triplets/versions/3 /content/drive/MyDrive/celeba_face_triplets

In [4]:
from torch.utils.data import DataLoader, Dataset
import os
from torchvision import transforms
import pandas as pd
from PIL import Image
class TripletDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir,'images')
        self.annotations = pd.read_csv(root_dir+'triplets.csv')
        self.transform = transform

    def __len__(self):
        return self.annotations.shape[0]

    def __getitem__(self,index):
        anchor_img = self.annotations.iloc[index,0]
        positive_img = self.annotations.iloc[index,2]
        negative_img = self.annotations.iloc[index,4]

        anc_img_path = os.path.join(self.image_dir, anchor_img)
        pos_img_path = os.path.join(self.image_dir, positive_img)
        neg_img_path = os.path.join(self.image_dir, negative_img)

        anc_image = Image.open(anc_img_path).convert("RGB")
        pos_image = Image.open(pos_img_path).convert("RGB")
        neg_image = Image.open(neg_img_path).convert("RGB")

        if self.transform:
            anc_image = self.transform(anc_image)
            pos_image = self.transform(pos_image)
            neg_image = self.transform(neg_image)
        return anc_image, pos_image, neg_image

In [5]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
path = '/content/drive/MyDrive/celeba_face_triplets/CelebA FR Triplets/CelebA FR Triplets/'
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
try:
    dataset = TripletDataset(root_dir=path, transform=transform)
    print(f"Found {len(dataset)} triplets.")
except Exception as e:
    print(f"Could not load dataset. Error: {e}")


train_indice, test_indice = train_test_split(list(range(len(dataset))),test_size=0.2, random_state=42)
train_subset, test_subset = Subset(dataset, train_indice), Subset(dataset, test_indice)
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True,  num_workers=4,pin_memory=True, prefetch_factor=2)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=True)
print(dataset[0][1].shape)


Found 16332 triplets.
torch.Size([3, 112, 112])


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class FaceRecognition(nn.Module):
    def __init__(self):
        super(FaceRecognition,self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3,out_channels=8, kernel_size=3, padding=1) #112*112
        self.bn1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16, kernel_size=3, padding=1) #56*56
        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(in_channels=16,out_channels=32, kernel_size=3, padding=1) #28*28
        self.bn3 = nn.BatchNorm2d(32)

        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) #14*14
        self.bn4 = nn.BatchNorm2d(64)

        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) #7*7
        self.bn5 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(in_features=128 * 3 * 3, out_features=512)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.normalize(x, p=2, dim=1)

        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
import torch.optim as optim
from numpy.linalg import norm as norm
import torch.nn as nn
model = FaceRecognition().to(device)
# def triplet_loss():
#     alpha = 0.2 #A hyperparameter
#     loss = norm(anc_vec - pos_vec)-norm(anc_vec-neg_vec)+alpha
#     return loss
NUM_EPOCHS = 25
criterion = nn.TripletMarginLoss(margin=0.2, p=2)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader) * NUM_EPOCHS)

In [8]:
NUM_EPOCHS = 25
model.train()
for epochs in range(NUM_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        anc_images, pos_images, neg_images = data[0].to(device), data[1].to(device), data[2].to(device)
        anc_embedding = model(anc_images)
        pos_embedding = model(pos_images)
        neg_embedding = model(neg_images)
        optimizer.zero_grad()

        loss = criterion(anc_embedding, pos_embedding, neg_embedding)
        loss.backward()

        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epochs + 1}/{NUM_EPOCHS}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training')

Epoch [1/25], Step [100/409], Loss: 0.1308
Epoch [1/25], Step [200/409], Loss: 0.1082
Epoch [1/25], Step [300/409], Loss: 0.0991
Epoch [1/25], Step [400/409], Loss: 0.0964
Epoch [2/25], Step [100/409], Loss: 0.0775
Epoch [2/25], Step [200/409], Loss: 0.0749
Epoch [2/25], Step [300/409], Loss: 0.0743
Epoch [2/25], Step [400/409], Loss: 0.0756
Epoch [3/25], Step [100/409], Loss: 0.0560
Epoch [3/25], Step [200/409], Loss: 0.0600
Epoch [3/25], Step [300/409], Loss: 0.0626
Epoch [3/25], Step [400/409], Loss: 0.0587
Epoch [4/25], Step [100/409], Loss: 0.0451
Epoch [4/25], Step [200/409], Loss: 0.0452
Epoch [4/25], Step [300/409], Loss: 0.0466
Epoch [4/25], Step [400/409], Loss: 0.0512
Epoch [5/25], Step [100/409], Loss: 0.0358
Epoch [5/25], Step [200/409], Loss: 0.0364
Epoch [5/25], Step [300/409], Loss: 0.0367
Epoch [5/25], Step [400/409], Loss: 0.0381
Epoch [6/25], Step [100/409], Loss: 0.0264
Epoch [6/25], Step [200/409], Loss: 0.0271
Epoch [6/25], Step [300/409], Loss: 0.0284
Epoch [6/25

In [9]:
torch.save(model.state_dict(), '/content/drive/MyDrive/celeba_face_triplets/my_model_weights.pth')

In [10]:
import torch

print("\n--- Running Simple Evaluation on Test Set ---")

model.eval()

total_loss = 0
correct_triplets = 0
total_triplets = 0

with torch.no_grad():
    for data in test_loader:
        anc_images, pos_images, neg_images = data[0].to(device), data[1].to(device), data[2].to(device)

        anc_embedding = model(anc_images)
        pos_embedding = model(pos_images)
        neg_embedding = model(neg_images)

        loss = criterion(anc_embedding, pos_embedding, neg_embedding)
        total_loss += loss.item()

        dist_pos = torch.norm(anc_embedding - pos_embedding, dim=1)
        dist_neg = torch.norm(anc_embedding - neg_embedding, dim=1)

        correct_triplets += torch.sum(dist_pos < dist_neg).item()
        total_triplets += anc_images.size(0)


avg_test_loss = total_loss / len(test_loader)
accuracy = 100 * correct_triplets / total_triplets

print(f"Average Test Loss: {avg_test_loss:.4f}")
print(f"Triplet Accuracy: {accuracy:.2f}%")



--- Running Simple Evaluation on Test Set ---
Average Test Loss: 0.0727
Triplet Accuracy: 85.49%


In [11]:
!cp /content/drive/MyDrive/celeba_face_triplets/my_model_weights.pth /content/drive/MyDrive/IntelligentSystem/my_model_weights.pth

In [12]:
!ls /content/drive/MyDrive/IntelligentSystem

IntelligentSystem.ipynb  my_model_weights.pth  TestModel.ipynb
